## Problem Statement:

The Taxi and Limousine Commission (TLC) of New York City collects trip record data from licensed taxis and for-hire vehicles (FHVs) and provides it to the public. The data includes details such as pick-up and drop-off times, locations, passenger counts, and payment information for each trip. As a data engineer, your task is to build a batch data processing pipeline using PySpark to process and analyze this data to gain insights into taxi and FHV trips in New York City.

### Goals:

Data ingestion: Download the trip record data from the NYC TLC website and ingest it into the pipeline for further processing.

Data cleaning and validation: Perform data quality checks and validation to ensure that the data is clean and consistent. Identify and remove duplicates, null values, and other data quality issues that may impact downstream analysis.

Data transformation: Transform the raw trip record data into a format that is optimized for analysis. This may include aggregating the data by time periods, geographical regions, and other factors of interest.

Data analysis: Use PySpark to perform statistical analysis, data exploration, and data visualization to gain insights into taxi and FHV trips in New York City. This may include identifying popular pick-up and drop-off locations, peak trip times, and other patterns and trends in the data.

Data storage: Store the processed and analyzed data in a suitable data storage system such as Hadoop Distributed File System (HDFS) or Apache Cassandra for future use.

Automation and scheduling: Automate the data processing pipeline using tools such as Apache Airflow or Apache Oozie. Schedule the pipeline to run at regular intervals to ensure that the data is up to date and accurate.

---

The overall goal of the project is to build a batch data processing pipeline using PySpark to extract insights from the NYC TLC trip record data. The pipeline should be scalable, efficient, and automated to enable easy data processing and analysis.

### Import Libraries and Intiate Spark session

In [1]:
# !pip install geospark
# !pip install pyspark

In [2]:
import configparser

import pandas as pd

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, TimestampType

import geospark

In [3]:
spark = SparkSession.builder.appName("nyc_batch_pipeline").getOrCreate()

23/05/16 12:02:00 WARN Utils: Your hostname, joker021-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/05/16 12:02:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/16 12:02:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

### Data Ingestion

In [5]:
# Parsing from config file
conf = configparser.ConfigParser()
conf.read("config")
data_source_path = conf.get("DATASOURCE PATH", "PATH")

# Schema
schema = StructType([
    StructField('vendor_name', StringType(), True), 
    StructField('Trip_Pickup_DateTime', StringType(), True), 
    StructField('Trip_Dropoff_DateTime', StringType(), True), 
    StructField('Passenger_Count', LongType(), True), 
    StructField('Trip_Distance', DoubleType(), True), 
    StructField('Rate_Code', DoubleType(), True), 
    StructField('store_and_forward', DoubleType(), True), 
    StructField('Payment_Type', StringType(), True), 
    StructField('Fare_Amt', DoubleType(), True), 
    StructField('surcharge', DoubleType(), True), 
    StructField('mta_tax', DoubleType(), True), 
    StructField('Tip_Amt', DoubleType(), True), 
    StructField('Tolls_Amt', DoubleType(), True), 
    StructField('Total_Amt', DoubleType(), True)]
)

# Reading the DataSource from PySpark
df_full = spark.read.schema(schema).parquet(data_source_path, schema=schema)

In [6]:
print(f"No of Partitons: {df_full.rdd.getNumPartitions()}")

No of Partitons: 4


In [7]:
frac = 0.01
df = df_full.sample(fraction=frac, seed=123)

In [8]:
# Schema
df.printSchema()

root
 |-- vendor_name: string (nullable = true)
 |-- Trip_Pickup_DateTime: string (nullable = true)
 |-- Trip_Dropoff_DateTime: string (nullable = true)
 |-- Passenger_Count: long (nullable = true)
 |-- Trip_Distance: double (nullable = true)
 |-- Rate_Code: double (nullable = true)
 |-- store_and_forward: double (nullable = true)
 |-- Payment_Type: string (nullable = true)
 |-- Fare_Amt: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- Tip_Amt: double (nullable = true)
 |-- Tolls_Amt: double (nullable = true)
 |-- Total_Amt: double (nullable = true)



### Data Cleaning And Validation

#### Missing Values

In [9]:
no_of_row = df.count()
print(f"No of Rows: {no_of_row}")
print(f"No of cols: {len(df.columns)}")

No of Rows: 140868
No of cols: 14


In [10]:
null_count = df.\
select(
    [F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]
).collect()[0]\
.asDict()
null_col_list = [c for c in null_count if null_count[c] > 0]

In [11]:
null_count

{'vendor_name': 0,
 'Trip_Pickup_DateTime': 0,
 'Trip_Dropoff_DateTime': 0,
 'Passenger_Count': 0,
 'Trip_Distance': 0,
 'Rate_Code': 140868,
 'store_and_forward': 140862,
 'Payment_Type': 0,
 'Fare_Amt': 0,
 'surcharge': 0,
 'mta_tax': 140868,
 'Tip_Amt': 0,
 'Tolls_Amt': 0,
 'Total_Amt': 0}

In [12]:
# We could see below three cols have huge no of Null Values
# The Rate_Code and mta_tax is completely null
# store_and_forward have few rows present
null_col_list = ["Rate_Code", "store_and_forward", "mta_tax"]
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in null_col_list]).show()

+---------+-----------------+-------+
|Rate_Code|store_and_forward|mta_tax|
+---------+-----------------+-------+
|   140868|           140862| 140868|
+---------+-----------------+-------+



In [13]:
# Checking no of Distinct values in Null having columns
df.select([F.countDistinct(F.col(c)).alias(c) for c in null_col_list]).show()

+---------+-----------------+-------+
|Rate_Code|store_and_forward|mta_tax|
+---------+-----------------+-------+
|        0|                1|      0|
+---------+-----------------+-------+



In [14]:
# Count of distinct values on store_and_forward, we could see the values are very small compared to total rows
df.select(F.col("store_and_forward")).groupBy('store_and_forward').count().show()

+-----------------+------+
|store_and_forward| count|
+-----------------+------+
|              0.0|     6|
|             null|140862|
+-----------------+------+



In [15]:
# Since the amount of null values is very large compared to total no of records we are dropping those columns
df_not_null = df.drop(*null_col_list)

In [16]:
df_not_null.printSchema()

root
 |-- vendor_name: string (nullable = true)
 |-- Trip_Pickup_DateTime: string (nullable = true)
 |-- Trip_Dropoff_DateTime: string (nullable = true)
 |-- Passenger_Count: long (nullable = true)
 |-- Trip_Distance: double (nullable = true)
 |-- Payment_Type: string (nullable = true)
 |-- Fare_Amt: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- Tip_Amt: double (nullable = true)
 |-- Tolls_Amt: double (nullable = true)
 |-- Total_Amt: double (nullable = true)



#### Data Transformation and DateTime

In [17]:
# Currently the DateTime col are in String format, will check the format
date_col = ["Trip_Pickup_DateTime", "Trip_Dropoff_DateTime"]
df_not_null.select(date_col).show(3)

+--------------------+---------------------+
|Trip_Pickup_DateTime|Trip_Dropoff_DateTime|
+--------------------+---------------------+
| 2009-01-03 11:05:27|  2009-01-03 11:10:55|
| 2009-01-23 00:27:00|  2009-01-23 00:35:00|
| 2009-01-18 16:49:45|  2009-01-18 16:56:34|
+--------------------+---------------------+
only showing top 3 rows



In [18]:
# We could see the format is %Y-%M-%D %H:%M:%s
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
date_dict_map = {date_c: F.to_timestamp(F.col(date_c)) for date_c in date_col}
df_time_parsed = df_not_null.withColumns(date_dict_map)

In [19]:
df_time_parsed.printSchema()

root
 |-- vendor_name: string (nullable = true)
 |-- Trip_Pickup_DateTime: timestamp (nullable = true)
 |-- Trip_Dropoff_DateTime: timestamp (nullable = true)
 |-- Passenger_Count: long (nullable = true)
 |-- Trip_Distance: double (nullable = true)
 |-- Payment_Type: string (nullable = true)
 |-- Fare_Amt: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- Tip_Amt: double (nullable = true)
 |-- Tolls_Amt: double (nullable = true)
 |-- Total_Amt: double (nullable = true)



In [20]:
# Adding Duration Columns
df_date_parsed = df_time_parsed.withColumn(
    "duration", 
    F.col("Trip_Dropoff_DateTime").cast("long") - F.col("Trip_Pickup_DateTime").cast("long")
)

#### Distinct Values

In [21]:
string_cols = [f.name for f in df_date_parsed.schema.fields if isinstance(f.dataType, F.StringType)]
distinct_count = df_date_parsed.select([F.countDistinct(F.col(c)).alias(c) for c in string_cols])
distinct_count_pd = distinct_count.pandas_api().transpose()
dist_cols = distinct_count_pd[distinct_count_pd[0]<50].index.tolist()

/home/joker021/.local/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
/home/joker021/.local/lib/python3.10/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_list` loads all data into the driver's memory. It should only be used if the resulting list is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [22]:
df_date_parsed.select([F.countDistinct(F.col(c)).alias(c) for c in dist_cols]).show()

+-----------+------------+
|vendor_name|Payment_Type|
+-----------+------------+
|          3|           6|
+-----------+------------+



In [23]:
# View Distinct Values
for c in dist_cols:
    df_date_parsed.select(c).distinct().show()

+-----------+
|vendor_name|
+-----------+
|        CMT|
|        VTS|
|        DDS|
+-----------+

+------------+
|Payment_Type|
+------------+
|   No Charge|
|        CASH|
|      Credit|
|        Cash|
|     Dispute|
|      CREDIT|
+------------+



#### Duplicates

In [24]:
# Tried distinct method and directly dropDup but both consumed lot of memory, 
# hence applying a tranformation and removing the duplicates which resulted in same result as direct dropDuplicates
# but less mem consumption

# We are concating all cols, and then checking dup based on concated col
df_drop_by_concat = df_date_parsed\
.withColumn("concat_cols", F.concat_ws("||", *df_date_parsed.columns))\
.dropDuplicates(["concat_cols"])\
.drop("concat_cols")

In [25]:
# Taking count after duplicates
cnt_after_drop = df_drop_by_concat.count()

# No of Duplciates dropped
no_of_row - cnt_after_drop

4

In [26]:
df_cleaned = df_drop_by_concat

### Data Validation

#### Schema Validation

In [27]:
# Defining Schema for Validation
validate_schema = StructType(
    [
        StructField('vendor_name', StringType(), True), 
        StructField('Trip_Pickup_DateTime', TimestampType(), True), 
        StructField('Trip_Dropoff_DateTime', TimestampType(), True), 
        StructField('Passenger_Count', LongType(), True), 
        StructField('Trip_Distance', DoubleType(), True), 
        StructField('Payment_Type', StringType(), True), 
        StructField('Fare_Amt', DoubleType(), True), 
        StructField('surcharge', DoubleType(), True), 
        StructField('Tip_Amt', DoubleType(), True), 
        StructField('Tolls_Amt', DoubleType(), True),
        StructField('Total_Amt', DoubleType(), True),
        StructField('duration', LongType(), True)
    ]
)

In [28]:
# Validate Schema
assert validate_schema == df_cleaned.schema, "schema is not valid"

#### Null Value Validation

In [29]:
# This method cosumes more memory hence commenting the below, It better to do one by one
# is_null_values = df_cleaned.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_cleaned.columns]).collect()[0].asDict()
# [col for col in is_null_values if is_null_values[col] > 0]

In [30]:
for c in df_cleaned.columns:
#     cnt = df_cleaned.select(c).where(F.col(c).isNull()).count()
#     Experimenting below
    cnt = df_cleaned.where(F.col(c).isNull()).select(c).count()
    if cnt > 0:
        print(c, cnt)
else:
    print("There No Null columns")

There No Null columns


In [31]:
#### Data Range Validation

In [32]:
# df_numeric_cols = [
#     f.name 
#     for f in df_cleaned.schema.fields 
#     if isinstance(f.dataType, DoubleType) or isinstance(f.dataType, LongType)
# ]
# bounds = {
#     c: dict(
#         zip(["q1", "q3"], df_cleaned.approxQuantile(c, [0.25, 0.75], 0.1))
#     )
#     for c in df_numeric_cols
# }

In [33]:
# for c in bounds:
#     iqr = bounds[c]['q3'] - bounds[c]['q1']
#     bounds[c]['lower'] = bounds[c]['q1'] - (iqr * 1.5)
#     bounds[c]['upper'] = bounds[c]['q3'] + (iqr * 1.5)
# print(bounds)

In [34]:
# df_outlier = df_cleaned.select(
#     "*",
#     *[
#         F.when(
#             F.col(c).between(bounds[c]['lower'], bounds[c]['upper']),
#             0
#         ).otherwise(1).alias(c+"_out") 
#         for c in df_numeric_cols
#     ]
# )

In [35]:
# df_outlier.select([F.sum(c).alias(c) for c in df_outlier.columns if c.endswith("out")]).show()

### Data Analysis

#### Data Aggregations

In [36]:
df_cleaned.printSchema()

root
 |-- vendor_name: string (nullable = true)
 |-- Trip_Pickup_DateTime: timestamp (nullable = true)
 |-- Trip_Dropoff_DateTime: timestamp (nullable = true)
 |-- Passenger_Count: long (nullable = true)
 |-- Trip_Distance: double (nullable = true)
 |-- Payment_Type: string (nullable = true)
 |-- Fare_Amt: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- Tip_Amt: double (nullable = true)
 |-- Tolls_Amt: double (nullable = true)
 |-- Total_Amt: double (nullable = true)
 |-- duration: long (nullable = true)



In [37]:
numeric_col = ["Passenger_Count", "Trip_Distance", "Fare_Amt", "surcharge", "Tip_Amt", "Tolls_Amt", "Total_Amt", "duration"]
df_per_vendor = df_cleaned.select("vendor_name",*numeric_col)\
.groupBy(F.col("vendor_name"))\
.agg(*[F.mean(F.col(c)).alias(c+"_avg") for c in numeric_col])

In [38]:
df_per_vendor.show()

+-----------+-------------------+------------------+-----------------+-------------------+------------------+-------------------+------------------+-----------------+
|vendor_name|Passenger_Count_avg| Trip_Distance_avg|     Fare_Amt_avg|      surcharge_avg|       Tip_Amt_avg|      Tolls_Amt_avg|     Total_Amt_avg|     duration_avg|
+-----------+-------------------+------------------+-----------------+-------------------+------------------+-------------------+------------------+-----------------+
|        CMT| 1.3143077092268778|2.5235094185480196|9.554908014013968|                0.0|0.4324191293145434|0.11141176100925422|  10.1014725613109|641.4716972239243|
|        VTS|  2.106358196351353| 2.555544341394518|9.383387592271744|0.32444709753505235|0.5146023033356143|0.11781663584875528|10.340923372595803|744.4733048934962|
|        DDS| 1.3511137162954279| 2.720890973036341|9.770267291910915|  0.338862837045721|0.4148722157092615|0.13438452520515828| 10.65838686987105|769.7600234466588

In [39]:
numeric_col = ["Passenger_Count", "Trip_Distance", "Fare_Amt", "surcharge", "Tip_Amt", "Tolls_Amt", "Total_Amt", "duration"]
df_per_hour = df_cleaned.select("Trip_Pickup_DateTime", *numeric_col)\
.groupBy(F.hour(F.col("Trip_Pickup_DateTime")).alias("Hour"))\
.agg(*[F.mean(F.col(c)).alias(c+"_avg") for c in numeric_col])

In [40]:
df_per_hour.show(3)

+----+-------------------+------------------+------------------+-------------------+-------------------+--------------------+------------------+-----------------+
|Hour|Passenger_Count_avg| Trip_Distance_avg|      Fare_Amt_avg|      surcharge_avg|        Tip_Amt_avg|       Tolls_Amt_avg|     Total_Amt_avg|     duration_avg|
+----+-------------------+------------------+------------------+-------------------+-------------------+--------------------+------------------+-----------------+
|  12| 1.6621863799283154|2.3294190561529287|  8.98279569892473|                0.0|0.42923685782556753|  0.1350089605734768|  9.55092443249702|708.0710872162485|
|  22| 1.7998421883219358| 2.719929642293529|  9.77783403471858|0.27373750657548657| 0.5339281956864806| 0.08145055234087327|10.666950289321417|663.6657022619673|
|   1| 1.8166666666666667| 3.021709595959597|10.258030303030312|0.26224747474747473| 0.5628308080808083|0.055479797979797976|11.138588383838394|686.8825757575758|
+----+----------------

In [41]:
numeric_col = ["Passenger_Count", "Trip_Distance", "Fare_Amt", "surcharge", "Tip_Amt", "Tolls_Amt", "Total_Amt", "duration"]
df_per_date = df_cleaned.select(
    F.to_date("Trip_Pickup_DateTime", "yyyy-MM-dd").alias("date"), 
    *numeric_col
)\
.groupBy("date")\
.agg(*[F.mean(F.col(c)).alias(c+"_avg") for c in numeric_col])

In [42]:
df_per_date.show(3)

+----------+-------------------+-----------------+-----------------+-------------------+-------------------+-------------------+------------------+-----------------+
|      date|Passenger_Count_avg|Trip_Distance_avg|     Fare_Amt_avg|      surcharge_avg|        Tip_Amt_avg|      Tolls_Amt_avg|     Total_Amt_avg|     duration_avg|
+----------+-------------------+-----------------+-----------------+-------------------+-------------------+-------------------+------------------+-----------------+
|2009-01-01| 1.8700092850510677|2.877926338594861|9.835369854534202|0.06499535747446611|0.30442587434230894|0.13930671618693907|10.353692355307954|685.7570411637264|
|2009-01-30| 1.6989327841228234|2.572048305560754| 9.75008238157648|0.21924733196030705| 0.5304474817449917| 0.1259127504212695|10.630557947949814|754.1759970043063|
|2009-01-22| 1.6182838813151563|2.543257818765037|9.633221732157178| 0.2118083400160385| 0.5555573376102646|0.12126303127506022|10.531273055332802| 678.713111467522|
+---

In [43]:
numeric_col = ["Passenger_Count", "Trip_Distance", "Fare_Amt", "surcharge", "Tip_Amt", "Tolls_Amt", "Total_Amt", "duration"]
df_per_week = df_cleaned.select("Trip_Pickup_DateTime", *numeric_col)\
.groupBy(F.weekofyear(F.col("Trip_Pickup_DateTime")).alias("WeekOfYear"))\
.agg(*[F.mean(F.col(c)).alias(c+"_avg") for c in numeric_col])

In [44]:
df_per_week.show(3)

+----------+-------------------+-----------------+-----------------+-------------------+-------------------+-------------------+------------------+-----------------+
|WeekOfYear|Passenger_Count_avg|Trip_Distance_avg|     Fare_Amt_avg|      surcharge_avg|        Tip_Amt_avg|      Tolls_Amt_avg|     Total_Amt_avg|     duration_avg|
+----------+-------------------+-----------------+-----------------+-------------------+-------------------+-------------------+------------------+-----------------+
|         1| 1.8763191503663372| 2.81293331988976|9.829425959534838|0.13201586341332258|0.38161053975935993|0.12571486186731176|10.472665860052409|698.9305639577872|
|         3| 1.6801956876245696|2.441457752008214|9.384307241650024|0.18131303980189648|0.47661412091562444|0.11642266111010416|10.158657063477659|696.8836141813131|
|         5|  1.663582871330907|2.486220196591955| 9.41574232168973|0.19807149416686506| 0.5031668990850651|0.10787150096935448|10.227199074861403|705.2540729907146|
+---

In [45]:
numeric_col = ["Passenger_Count", "Trip_Distance", "Fare_Amt", "surcharge", "Tip_Amt", "Tolls_Amt", "Total_Amt", "duration"]
df_per_dayofmonth = df_cleaned.select("Trip_Pickup_DateTime", *numeric_col)\
.groupBy(F.dayofmonth(F.col("Trip_Pickup_DateTime")).alias("DayOfMonth"))\
.agg(*[F.mean(F.col(c)).alias(c+"_avg") for c in numeric_col])

In [46]:
df_per_dayofmonth.show(3)

+----------+-------------------+------------------+----------------+-------------------+-------------------+-------------------+-----------------+-----------------+
|DayOfMonth|Passenger_Count_avg| Trip_Distance_avg|    Fare_Amt_avg|      surcharge_avg|        Tip_Amt_avg|      Tolls_Amt_avg|    Total_Amt_avg|     duration_avg|
+----------+-------------------+------------------+----------------+-------------------+-------------------+-------------------+-----------------+-----------------+
|        31| 1.8038999264164828|2.4890231788079467|9.26937086092716|0.11966519499632082| 0.4307763061074319|0.06725533480500369|9.887067696835917|657.0198675496689|
|        28| 1.6293159609120522|2.3341628664495118|9.11593919652552|0.21758957654723127| 0.5111248642779588|0.08086427795874053|9.934855591748102|722.7463626492943|
|        27| 1.6310228233305157|2.4156635672020306|9.16749788672866| 0.2092138630600169|0.49590659340659343| 0.1177726120033813|9.990390955198654|684.5978444632291|
+---------

In [47]:
numeric_col = ["Passenger_Count", "Trip_Distance", "Fare_Amt", "surcharge", "Tip_Amt", "Tolls_Amt", "Total_Amt", "duration"]
df_per_dayofweek = df_cleaned.select("Trip_Pickup_DateTime", *numeric_col)\
.groupBy(F.dayofweek(F.col("Trip_Pickup_DateTime")).alias("dayofweek"))\
.agg(*[F.mean(F.col(c)).alias(c+"_avg") for c in numeric_col])

In [48]:
df_per_dayofweek.show(3)

+---------+-------------------+------------------+-----------------+-------------------+-------------------+-------------------+------------------+-----------------+
|dayofweek|Passenger_Count_avg| Trip_Distance_avg|     Fare_Amt_avg|      surcharge_avg|        Tip_Amt_avg|      Tolls_Amt_avg|     Total_Amt_avg|     duration_avg|
+---------+-------------------+------------------+-----------------+-------------------+-------------------+-------------------+------------------+-----------------+
|        1| 1.8005262191764058|2.8216930184176694|9.700611270880483|0.11185216912439576| 0.4656758245120234|0.12108242060821126| 10.40087376858593|678.7389708131922|
|        6| 1.7117979797979799| 2.520908686868691| 9.51842747474747|0.22151515151515153|0.47601575757575765|0.11997777777777746|10.336986666666675| 710.664808080808|
|        3| 1.6393324061196106|2.5091883171070943| 9.38501752433936|0.20845618915159944|0.48296856745479827|0.12769346314325428|10.204135744089022|692.8576912378303|
+---

In [49]:
df_cnt_per_payment_type = df_cleaned.select("Payment_Type")\
.groupBy(F.col("Payment_Type"))\
.count()

In [50]:
df_cnt_per_payment_type.show()

+------------+-----+
|Payment_Type|count|
+------------+-----+
|   No Charge|  374|
|        CASH|60285|
|      Credit|28750|
|        Cash|49810|
|     Dispute|   68|
|      CREDIT| 1577|
+------------+-----+



In [51]:
numeric_col = ["Passenger_Count", "Trip_Distance", "Fare_Amt", "surcharge", "Tip_Amt", "Tolls_Amt", "Total_Amt", "duration"]
df_per_payment_type = df_cleaned.select("Payment_Type",*numeric_col)\
.groupBy(F.col("Payment_Type"))\
.agg(*[F.mean(F.col(c)).alias(c+"_avg") for c in numeric_col])

In [52]:
df_per_payment_type.show()

+------------+-------------------+------------------+------------------+-------------------+--------------------+-------------------+------------------+-----------------+
|Payment_Type|Passenger_Count_avg| Trip_Distance_avg|      Fare_Amt_avg|      surcharge_avg|         Tip_Amt_avg|      Tolls_Amt_avg|     Total_Amt_avg|     duration_avg|
+------------+-------------------+------------------+------------------+-------------------+--------------------+-------------------+------------------+-----------------+
|   No Charge| 1.2379679144385027|2.2401069518716574|10.134224598930484|                0.0| 0.03617647058823529|0.06914438502673798|10.239545454545457|698.6737967914438|
|        CASH| 2.0316330762212824|  2.38219228663847| 8.872720079621816| 0.3229161482955959|                 0.0|0.08728705316413718| 9.283686323297683|661.1249232810816|
|      Credit| 1.6828521739130435| 3.219395826086955|11.400533217391295|0.17965217391304347|   2.183159304347825|0.22185286956521838|13.988327999

In [53]:
df_cnt_payment_cnt_per_vendor = df_cleaned\
.select("vendor_name", "Payment_Type")\
.groupBy("vendor_name", "Payment_Type")\
.count()

In [54]:
df_cnt_payment_cnt_per_vendor.show()

+-----------+------------+-----+
|vendor_name|Payment_Type|count|
+-----------+------------+-----+
|        VTS|        CASH|53332|
|        DDS|      CREDIT| 1577|
|        CMT|      Credit|13399|
|        DDS|        CASH| 6953|
|        CMT|   No Charge|  374|
|        CMT|        Cash|49810|
|        VTS|      Credit|15351|
|        CMT|     Dispute|   68|
+-----------+------------+-----+

